<b>Data Science Capstone Progect, Week 3</b>

<b>Part I: Retrive Postal Code table from wiki page: <i>List of postal codes of Canada: M</i></b>
1. Use BeautifulSoup library for HTML page parsing:

In [1]:
!pip install beautifulsoup4

In [5]:
from bs4 import BeautifulSoup as bs

2. Take a page by URL and extract table with class="neighborhoods.shape"

In [42]:
r=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=bs(r.text, 'lxml')
table=soup.find('table', class_='neighborhoods.shape')

3. Take three colums from the table and clean the data:

In [60]:
columns=['PostalCode', 'Borough', 'Neighbourhood']
neighborhoods = pd.DataFrame(columns=columns)

i=0
for td in table.find_all('td'):
    if i==0:
        code=td.text.rstrip()
        i=1
    elif i==1:
        borough=td.text.rstrip()
        i=2
    elif i==2:
        neighbourhood=td.text.rstrip()
        if not borough.startswith('Not assigned'):
            if neighbourhood.startswith('Not assigned'):
                neighbourhood=borough
            neighborhoods=neighborhoods.append(
                {'PostalCode': code,
                'Borough': borough,
                'Neighbourhood': neighbourhood},
                ignore_index=True
            )
        i=0

4. Group by PostalCode and Borough with concatenation of duplicate Neighbourhoods:

In [61]:
neighborhoods=neighborhoods.groupby(['PostalCode', 'Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])
neighborhoods=neighborhoods.reset_index()
neighborhoods.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [62]:
neighborhoods.shape

(103, 3)

<b>Part II: Enrich the datased with latitude and the longitude coordinates</b>
1. Take geodata from URL:

In [79]:
geodata=pd.read_csv('https://cocl.us/Geospatial_data')
geodata.set_index(['Postal Code'], inplace=True)
geodata.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


2. Enrich previous dataset with geospatial data:

In [84]:
for index,row in neighborhoods.iterrows():
    coords = geodata.loc[row['PostalCode']]
    neighborhoods.at[index, 'Latitude'] = coords['Latitude']
    neighborhoods.at[index, 'Longtitude'] = coords['Longitude']

neighborhoods.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longtitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648
